<div style="align: center; margin: 0; padding: 0; height: 250px;">
    <br>
    <img src="https://www.nyc.gov/assets/tlc/images/content/hero/MRP-Closing-Week.jpg" style="display:block; margin:auto; width:65%; height:100%;">
</div><br><br> 

<div style="letter-spacing:normal; opacity:1.;">
<!--   https://xkcd.com/color/rgb/   -->
  <p style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; 
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>TLC Trip Record Data</strong></p>  
  
  <p style="text-align:center; background-color:romance; color: Jaguar; border-radius:10px; font-family:monospace; 
            line-height:1.4; font-size:22px; font-weight:normal; text-transform: capitalize; padding: 5px;"
     >Machine Learning Module: MONITORING - Ride Duration Prediction<br>using Regression Analysis<br></p><br>
    
  <div style="align: center;">
  <table style="text-align: center; background-color: romance; color: Jaguar; border-radius: 10px; font-family: monospace;
                  line-height:1.4; font-size: 21px; font-weight: normal; text-transform: capitalize; padding: 5px; 
                  margin: 0 auto;">
    <tr><td style="text-align: left; padding-left: 0px;"
            > MONITORING <span style="font-size: 16px;">(to ensure that deployed models remain accurate,<br>reliable, and aligned with business objectives over time)</span></td></tr>
    <tr><td style="text-align: left; padding-left: 0px;"
            > MLOps <span style="font-size: 16px;">(CI/CD, Model Versioning, Monitoring, Automated Retraining,<br>Security, Scalability, Collaboration)</span></td></tr>
  </table>
  </div>

</div>

[Introduction to MLOps Monitoring](https://github.com/dimzachar/mlops-zoomcamp/blob/master/notes/Week_5/intro.md)

**Dataset Info**


**Context**

Yellow and green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers authorized under the Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP). The trip data was not created by the TLC, and TLC makes no representations as to the accuracy of these data.

For-Hire Vehicle (“FHV”) trip records include fields capturing the dispatching base license number and the pick-up date, time, and taxi zone location ID (shape file below). These records are generated from the FHV Trip Record submissions made by bases. Note: The TLC publishes base trip record data as submitted by the bases, and we cannot guarantee or confirm their accuracy or completeness. Therefore, this may not represent the total amount of trips dispatched by all TLC-licensed bases. The TLC performs routine reviews of the records and takes enforcement actions when necessary to ensure, to the extent possible, complete and accurate information.


**ATTENTION!**

On 05/13/2022, we are making the following changes to trip record files:

- All files will be stored in the PARQUET format. Please see the ‘Working With PARQUET Format’ under the Data Dictionaries and MetaData section.
- Trip data will be published monthly (with two months delay) instead of bi-annually.
- HVFHV files will now include 17 more columns (please see High Volume FHV Trips Dictionary for details). Additional columns will be added to the old files as well. The earliest date to include additional columns: February 2019.
- Yellow trip data will now include 1 additional column (‘airport_fee’, please see Yellow Trips Dictionary for details). The additional column will be added to the old files as well. The earliest date to include the additional column: January 2011.


**Download the data for March 2023**

Dataset: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page


**Data Dictionaries and MetaData**

- We'll use the same `NYC taxi dataset`, but instead of "Yellow Taxi Trip Records", we'll use `"Green Taxi Trip Records"`.

> `Green Trips Data Dictionary`: https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

**TASK**


In this homework, we'll take the ride duration prediction model that we deployed in batch mode in homework 4 and improve the reliability of our code with unit and integration tests. 

[homework](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2023/06-best-practices/homework.md)



**Table of Content**


1. Import Libraries and Ingest Data 
2. Docker 
3. Baseline Model nyc_taxi Data
    - Q1. 
    

<div style="letter-spacing:normal; opacity:1.;">
  <h1 style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; border-radius:20px;
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>1. Import Libraries & Ingest Data</strong></h1>   
</div>

> ⚠️ Not Recommended conda `base` env, work on `venv`

- https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf

```
pip freeze > requirements.txt
conda list -e > requirements.txt

# new conda virtual environment
conda create --name "pytest-ops" python=3.10 jupyter -y
conda activate "pytest-ops"

# install all package dependencies
pip install -r requirements.txt
conda install -c conda-forge --file=requirements.txt      # mostly not work
conda install -c conda-forge pandas==2.0.2 -q -y

# if The environment is inconsistent, try below
conda update -n base -c defaults conda --force-reinstall
conda install anaconda --force-reinstall

```
- pipenv
```sh
pip install pipenv
pipenv shell        # or pipenv run
pipenv --where
pipenv --venv
```

**You must use the `--no-deps` option in the pip install command in order to avoid bundling dependencies into your conda-package.**

If you run pip install without the `--no-deps` option, pip will often install dependencies in your conda recipe and those dependencies will become part of your package. This wastes space in the package and `increases the risk of file overlap`, file clobbering, and broken packages.

There might be cases where you want to install a package directly from a local directory or a specific location, without relying on the package indexes. In such situations, you can use the `--no-index` option to tell pip not to look for the package in any indexes.

```
- command1 & command2  # runs simultaneously
- command1 ; command2  # runs sequentially
- command1 && command2 # runs sequentially, runs command2 only if command1 succeeds
- command1 || command2 # runs sequentially, runs command2 only if command1 fails
```

In [1]:
%%writefile requirements.txt 
# To get started with MLflow you'll need to install the appropriate Python package.

# for parquet file
pyarrow==11.0.0
fastparquet==2023.4.0

pandas==2.0.2
matplotlib==3.7.1
orjson==3.8.8          # orjson is a fast, correct JSON library
tqdm==4.65.0
# requests==2.29.0

# ML Model packages
scikit-learn==1.2.2

# MLOPS packages
mlflow==2.3.1
prefect==2.10.18
prefect-email==0.2.2

# MLOPS Cloud packages
boto3~=1.24.28
prefect-aws==0.3.4

# Monitoring
joblib==1.2.0
evidently==0.3.3
psycopg==3.1.9
# psycopg[binary]==3.1.9

# Optionally
pipenv
jupyter
ipykernel
ipywidgets

Writing requirements.txt


In [2]:
import os, sys, platform, IPython.display

# !{sys.executable} -m pip install -Uq -r requirements.txt  #  --no-deps --no-cache-dir --force-reinstall --no-index
!jupyter nbextension enable --py widgetsnbextension

# IPython.display.clear_output()
print("Python  :", sys.version)
print("Platform:", platform.system(), platform.platform())
print("Actv Env:", os.environ['CONDA_DEFAULT_ENV'])

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Python  : 3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]
Platform: Linux Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
Actv Env: base


In [3]:
import warnings
from numba import NumbaDeprecationWarning
# Suppress NumbaDeprecationWarning in umap.distances module
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

import os
# TensorFlow to only display error messages 
# and suppress warnings and informational messages.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy
from scipy.stats import stats

import os
import sys
import boto3
import click
import pickle
import datetime
# import pathlib
# import argparse
import requests
# import urllib.request
from glob import glob

# Import joblib for model persistence and tqdm for progress bars
from joblib import load, dump
# from tqdm import tqdm           # console-based
# from tqdm.notebook import tqdm  # jupyter-based
from tqdm.auto import tqdm        # automatically selects
# tqdm._instances.clear()

# Import sklearn for machine learning
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# import mlflow
# import wandb
# import prefect
# from prefect import task, flow, Flow
# from prefect.tasks import task_input_hash
# from prefect.artifacts import create_markdown_artifact

# Import Evidently for data drift and model performance monitoring
# from evidently import ColumnMapping
# from evidently.report import Report
# from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric
# from evidently.metrics import DatasetMissingValuesMetric, ColumnQuantileMetric

# memory management performs garbage collection 
import gc
gc.collect()

8

<div style="letter-spacing:normal; opacity:1.;">
  <h1 style="text-align:center; background-color: lightsalmon; color: Jaguar; border-radius:10px; font-family:monospace; border-radius:20px;
            line-height:1.4; font-size:32px; font-weight:bold; text-transform: uppercase; padding: 9px;">
            <strong>2. Recognizing and Understanding Data</strong></h1>   
</div>

## Ingest Data [wget](https://linuxways.net/centos/linux-wget-command-with-examples/) or [curl](https://daniel.haxx.se/blog/2020/09/10/store-the-curl-output-over-there/)

In [4]:
# "Yellow Taxi Trip Records" Download the data for January, February 2022
# !wget -N -P "./data" https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet -q
!wget -N -P "./data" https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet -q

In [5]:
glob(f'./data/*.parquet')

['./data/yellow_tripdata_2022-02.parquet']

In [6]:
import os
# Get the current working directory
# current_dir = os.getcwd()

# Create a new directory for storing MLflow data
os.makedirs('./pycode', exist_ok=True)
os.makedirs('./tests', exist_ok=True)
# os.makedirs('./data', exist_ok=True)
# os.makedirs('./output', exist_ok=True)
# os.makedirs('./models', exist_ok=True)
# os.makedirs('./config', exist_ok=True)

# Docker

In [7]:
%%writefile Dockerfile

FROM python:3.10.0-slim

RUN pip install -U pip & pip install pipenv

COPY [ "Pipfile", "Pipfile.lock", "./" ]

RUN pipenv install --system --deploy

COPY [ "pycode/batch.py", "pycode/batch.py" ]
COPY [ "models/model.bin", "models/model.bin" ]

ENTRYPOINT [ "python", "pycode/batch.py" ]

Writing Dockerfile


# Baseline Model nyc_taxi Data

## Q1. Refactoring

Before we can start converting our code with tests, we need to 
refactor it. We'll start by getting rid of all the global variables. 

* Let's create a function `main` with two parameters: `year` and
`month`.
* Move all the code (except `read_data`) inside `main`
* Make `categorical` a parameter for `read_data` and pass it inside `main`

Now we need to create the "main" block from which we'll invoke
the main function. How does the `if` statement that we use for
this looks like? 


Hint: after refactoring, check that the code still works. Just run
it e.g. for Feb 2022 and see if it finishes successfully. 

To make it easier to run it, you can write results to your local
filesystem. E.g. here:

```python
output_file = f'taxi_type=yellow_year={year:04d}_month={month:02d}.parquet'
```

In [8]:
%%writefile ./pycode/batch.py
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import pickle
from tqdm.auto import tqdm
tqdm._instances.clear()

import numpy as np
import pandas as pd


def prepare_data(df, categorical) -> pd.DataFrame:
    df["tpep_dropoff_datetime"] = pd.to_datetime(df.tpep_dropoff_datetime)
    df["tpep_pickup_datetime"] = pd.to_datetime(df.tpep_pickup_datetime)

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df['duration'] = df['duration'].dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # df[categorical] = df[categorical].astype(str)
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


def read_data(filename, categorical) -> pd.DataFrame:
    S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')

    """Read data into DataFrame"""
    if S3_ENDPOINT_URL is not None:
        options = {
            'client_kwargs': {
                'endpoint_url': S3_ENDPOINT_URL
            }
        }

        df = pd.read_parquet(filename, storage_options=options)
    else:
        df = pd.read_parquet(filename)

    return prepare_data(df, categorical)


def predict_duration(df: pd.DataFrame, categorical, dv, lr) -> np.ndarray:
    """Predict the duration using the trained model"""
    dicts  = df[categorical].to_dict(orient='records')
    X_val  = dv.transform(dicts)
    y_pred = lr.predict(X_val)
    return y_pred

        
def save_results(df: pd.DataFrame, y_pred: np.ndarray, output_file: str) -> None:
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred    
    S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')   

    """Save the predicted results to a parquet file""" 
    if S3_ENDPOINT_URL is not None:
        options = {
            'client_kwargs': {
                'endpoint_url': S3_ENDPOINT_URL
            }
        }

        df.to_parquet(output_file, engine='pyarrow', index=False, storage_options=options)
    else:
        os.makedirs('output', exist_ok=True)        
        df_result.to_parquet(        
            output_file,
            engine='pyarrow',
            # compression=None,
            index=False,
            storage_options=None
        )
    return None 

    
def main(year, month):
    categorical = ['PULocationID', 'DOLocationID']
    # input_file  = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'   
    input_file  = f'./data/yellow_tripdata_{year:04d}-{month:02d}.parquet' 
    output_file = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'
  
    steps = ["Loading model", "Reading data", "Predict data"]
    with tqdm(total=len(steps), desc="Running steps", leave=True) as pbar:
        # Step 1: Loading model
        pbar.set_description(steps[0])
        with open('models/model.bin', 'rb') as f_in:
            dv, lr = pickle.load(f_in)
        pbar.update(1)

        # Step 2: Reading data
        pbar.set_description(steps[1])
        df = read_data(input_file, categorical)
        df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
        pbar.update(1)
        
        # Step 3: Predict data
        pbar.set_description(steps[2])
        y_pred = predict_duration(df, categorical, dv, lr)
        pbar.update(1)
        pbar.close()    
        

    # Print Prediction
    print('predicted mean duration:', y_pred.mean().round(2))
    print('predicted sum duration:', y_pred.sum().round(2))

    # save_results
    save_results(df, y_pred, output_file)

    
if __name__ == '__main__':    
    # Global Parameters
    year        = int(sys.argv[1]) # 2022
    month       = int(sys.argv[2]) # 2 
    
    main(year, month)

Writing ./pycode/batch.py


In [9]:
!python ./pycode/batch.py 2022 2

Predict data: 100%|███████████████████████████████| 3/3 [01:38<00:00, 32.96s/it]
predicted mean duration: 12.51
predicted sum duration: 36516505.75


## Q2. Installing pytest

Now we need to install `pytest`:

```bash
pipenv install --dev pytest
```

Next, create a folder `tests` and then two files inside. 

The first one will be the file with tests. We can name it `test_batch.py`. 

The second file will be `__init__.py`. So, why do we need this second file?

- To define a package and specify its boundaries
- To manage the import of modules from the package 
- Both of the above options are correct
- To initialize a new object

In [11]:
%%writefile ./tests/__init__.py
 

Writing ./tests/__init__.py


In Python, the __init__.py file is a special file that is used to mark a directory as a Python package. When a directory contains an __init__.py file, Python recognizes it as a package and allows the directory to be treated as a module.

Here are a few key points about the __init__.py file:

1. Package Initialization: The primary purpose of the __init__.py file is to initialize the package when it is imported. It can contain Python code that sets up the package's environment, imports necessary modules, or performs other initialization tasks. This code is executed automatically when the package is imported.

2. Namespace Definition: The __init__.py file also defines the package's namespace. It specifies which modules and sub-packages are part of the package and can be accessed when the package is imported. By importing specific modules in the __init__.py file, you can control the names available to users when they import the package.

3. Python Package Structure: The presence of __init__.py files in directories helps define the structure of a Python package. It allows for hierarchical organization by enabling the creation of sub-packages within a package. Each sub-package can have its own __init__.py file, further defining its namespace and initialization logic.

4. Compatibility: The use of __init__.py is a convention that has been established in Python to define packages. By adhering to this convention, developers ensure that their code is compatible with Python's import system and can be easily understood by other Python developers.

Overall, the __init__.py file is crucial in Python packages as it enables proper package initialization, defines namespaces, and contributes to the organization and structure of a package. It helps ensure modularity, reusability, and maintainability in Python projects.

## Q3. Writing first unit test

Now let's cover our code with unit tests.

We'll start with the pre-processing logic inside `read_data`.

It's difficult to test right now because first reads
the file and then performs some transformations. We need to split this 
code into two parts: reading (I/O) and transformation. 

So let's create a function `prepare_data` that takes in a dataframe 
(and some other parameters too) and applies some transformation to it.

(That's basically the entire `read_data` function after reading 
the parquet file)

Now create a test and use this as input:

```python
data = [
    (None, None, dt(1, 2), dt(1, 10)),
    (1, None, dt(1, 2), dt(1, 10)),
    (1, 2, dt(2, 2), dt(2, 3)),
    (None, 1, dt(1, 2, 0), dt(1, 2, 50)),
    (2, 3, dt(1, 2, 0), dt(1, 2, 59)),
    (3, 4, dt(1, 2, 0), dt(2, 2, 1)),     
]

columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']
df = pd.DataFrame(data, columns=columns)
```

Where `dt` is a helper function:

```python
from datetime import datetime

def dt(hour, minute, second=0):
    return datetime(2022, 1, 1, hour, minute, second)
```

Define the expected output and use the assert to make sure 
that the actual dataframe matches the expected one

Tip: When you compare two Pandas DataFrames, the result is also a DataFrame.
The same is true for Pandas Series. Also, a DataFrame could be turned into a
list of dictionaries.  

How many rows should be there in the expected dataframe?

- 1
- 2
- 3
- 4

In [14]:
%%writefile ./tests/test_batch.py
#!/usr/bin/env python
# coding: utf-8

import os
import sys

# Append the parent directory of the tests folder to the system path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from pycode import batch
from datetime import datetime

import pickle
from tqdm.auto import tqdm
tqdm._instances.clear()

import numpy as np
import pandas as pd
from pandas import Timestamp
import pandas.testing as pd_testing


def dt(hour, minute, second=0):
    return datetime(2022, 1, 1, hour, minute, second)


def test_read_data() -> pd.DataFrame:
    """Read data into DataFrame"""
    data    = [
        (None, None, dt(1, 2), dt(1, 10)),
        (1, None, dt(1, 2), dt(1, 10)),
        (1, 2, dt(2, 2), dt(2, 3)),
        (None, 1, dt(1, 2, 0), dt(1, 2, 50)),
        (2, 3, dt(1, 2, 0), dt(1, 2, 59)),
        (3, 4, dt(1, 2, 0), dt(2, 2, 1)),     
    ]
    categorical = ['PULocationID', 'DOLocationID']   
    columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']

    df = pd.DataFrame(data, columns=columns)

    df["tpep_dropoff_datetime"] = pd.to_datetime(df.tpep_dropoff_datetime)
    df["tpep_pickup_datetime"] = pd.to_datetime(df.tpep_pickup_datetime)

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df['duration'] = df['duration'].dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # df[categorical] = df[categorical].astype(str)
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    print(df)

    actual_features = df
    # print(df.to_dict())
    
    expected_data = {
        'PULocationID': {0: '-1', 1: '1', 2: '1'}, 
        'DOLocationID': {0: '-1', 1: '-1', 2: '2'}, 
        'tpep_pickup_datetime': {0: Timestamp('2022-01-01 01:02:00'), 1: Timestamp('2022-01-01 01:02:00'), 2: Timestamp('2022-01-01 02:02:00')}, 
        'tpep_dropoff_datetime': {0: Timestamp('2022-01-01 01:10:00'), 1: Timestamp('2022-01-01 01:10:00'), 2: Timestamp('2022-01-01 02:03:00')}, 
        'duration': {0: 8.0, 1: 8.0, 2: 1.0}
    }
    expected_features = pd.DataFrame(expected_data)

    assert actual_features.equals(expected_features)
    assert (actual_features == expected_features).all().all()
    # pd_testing.assert_frame_equal(actual_features, expected_features)

    return df

    
if __name__ == '__main__':
    test_read_data()

Overwriting ./tests/test_batch.py


In [15]:
!python ./tests/test_batch.py

  PULocationID DOLocationID  ... tpep_dropoff_datetime duration
0           -1           -1  ...   2022-01-01 01:10:00      8.0
1            1           -1  ...   2022-01-01 01:10:00      8.0
2            1            2  ...   2022-01-01 02:03:00      1.0

[3 rows x 5 columns]


## Q4. Mocking S3 with Localstack 

Now let's prepare for an integration test. In our script, we 
write data to S3. So we'll use Localstack to mimic S3.

First, let's run Localstack with Docker compose. Let's create a 
`docker-compose.yaml` file with just one service: localstack. Inside
localstack, we're only interested in running S3. 

Start the service and test it by creating a bucket where we'll
keep the output. Let's call it "nyc-duration".

With AWS CLI, this is how we create a bucket:

```bash
aws s3 mb s3://nyc-duration
```

Then we need to check that the bucket was successfully created. With AWS, this is how we typically do it:

```bash
aws s3 ls
```

In both cases we should adjust commands for localstack. Which option do we need to use for such purposes?

- `--endpoint-url`
- `--profile`
- `--region`
- `--version`


## Make input and output paths configurable

Right now the input and output paths are hardcoded, but we want
to change it for the tests. 

One of the possible ways would be to specify `INPUT_FILE_PATTERN` and `OUTPUT_FILE_PATTERN` via the env 
variables. Let's do that:


```bash
export INPUT_FILE_PATTERN="s3://nyc-duration/in/{year:04d}-{month:02d}.parquet"
export OUTPUT_FILE_PATTERN="s3://nyc-duration/out/{year:04d}-{month:02d}.parquet"
```

And this is how we can read them:

```python
def get_input_path(year, month):
    default_input_pattern = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    input_pattern = os.getenv('INPUT_FILE_PATTERN', default_input_pattern)
    return input_pattern.format(year=year, month=month)


def get_output_path(year, month):
    default_output_pattern = 's3://nyc-duration-prediction-alexey/taxi_type=fhv/year={year:04d}/month={month:02d}/predictions.parquet'
    output_pattern = os.getenv('OUTPUT_FILE_PATTERN', default_output_pattern)
    return output_pattern.format(year=year, month=month)


def main(year, month):
    input_file = get_input_path(year, month)
    output_file = get_output_path(year, month)
    # rest of the main function ... 
```


## Reading from Localstack S3 with Pandas

So far we've been reading parquet files from S3 with using
pandas `read_parquet`. But this way we read it from the
actual S3 service. Now we need to replace it with our localstack
one.

For that, we need to specify the endpoint url:

```python
options = {
    'client_kwargs': {
        'endpoint_url': S3_ENDPOINT_URL
    }
}

df = pd.read_parquet('s3://bucket/file.parquet', storage_options=options)
```

Let's modify our `read_data` function:

- check if `S3_ENDPOINT_URL` is set, and if it is, use it for reading
- otherwise use the usual way

In [16]:
%%writefile ./pycode/batch.py
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import pickle
from tqdm.auto import tqdm
tqdm._instances.clear()

import numpy as np
import pandas as pd


def prepare_data(df, categorical) -> pd.DataFrame:
    df["tpep_dropoff_datetime"] = pd.to_datetime(df.tpep_dropoff_datetime)
    df["tpep_pickup_datetime"] = pd.to_datetime(df.tpep_pickup_datetime)

    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df['duration'] = df['duration'].dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # df[categorical] = df[categorical].astype(str)
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    return df


def read_data(filename, categorical) -> pd.DataFrame:
    S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')

    """Read data into DataFrame"""
    if S3_ENDPOINT_URL is not None:
        options = {
            'client_kwargs': {
                'endpoint_url': S3_ENDPOINT_URL
            }
        }

        df = pd.read_parquet(filename, storage_options=options)
    else:
        df = pd.read_parquet(filename)

    return prepare_data(df, categorical)


def predict_duration(df: pd.DataFrame, categorical, dv, lr) -> np.ndarray:
    """Predict the duration using the trained model"""
    dicts  = df[categorical].to_dict(orient='records')
    X_val  = dv.transform(dicts)
    y_pred = lr.predict(X_val)
    return y_pred

        
def save_results(df: pd.DataFrame, y_pred: np.ndarray, output_file: str) -> None:
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred    
    S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')   

    """Save the predicted results to a parquet file""" 
    if S3_ENDPOINT_URL is not None:
        options = {
            'client_kwargs': {
                'endpoint_url': S3_ENDPOINT_URL
            }
        }

        df.to_parquet(output_file, engine='pyarrow', index=False, storage_options=options)
    else:
        os.makedirs('output', exist_ok=True)        
        df_result.to_parquet(        
            output_file,
            engine='pyarrow',
            # compression=None,
            index=False,
            storage_options=None
        )
    return None 


def get_input_path(year, month):
    # default_input_pattern = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    default_input_pattern  = f'./data/yellow_tripdata_{year:04d}-{month:02d}.parquet'

    input_pattern = os.getenv('INPUT_FILE_PATTERN', default_input_pattern)
    return input_pattern.format(year=year, month=month)


def get_output_path(year, month):
    # default_output_pattern = f's3://nyc-duration-prediction-alexey/taxi_type=fhv/year={year:04d}/month={month:02d}/predictions.parquet'
    default_output_pattern = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet' 

    output_pattern = os.getenv('OUTPUT_FILE_PATTERN', default_output_pattern)
    return output_pattern.format(year=year, month=month)

    
def main(year, month):
    categorical = ['PULocationID', 'DOLocationID']
    input_file = get_input_path(year, month)
    output_file = get_output_path(year, month)
  
    steps = ["Loading model", "Reading data", "Predict data"]
    with tqdm(total=len(steps), desc="Running steps", leave=True) as pbar:
        # Step 1: Loading model
        pbar.set_description(steps[0])
        with open('models/model.bin', 'rb') as f_in:
            dv, lr = pickle.load(f_in)
        pbar.update(1)

        # Step 2: Reading data
        pbar.set_description(steps[1])
        df = read_data(input_file, categorical)
        df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
        pbar.update(1)
        
        # Step 3: Predict data
        pbar.set_description(steps[2])
        y_pred = predict_duration(df, categorical, dv, lr)
        pbar.update(1)
        pbar.close()    
        

    # Print Prediction
    print('predicted mean duration:', y_pred.mean().round(2))
    print('predicted sum duration:', y_pred.sum().round(2))

    # save_results
    save_results(df, y_pred, output_file)

    
if __name__ == '__main__':    
    # Global Parameters
    year        = int(sys.argv[1]) # 2022
    month       = int(sys.argv[2]) # 2 
    
    main(year, month)

Overwriting ./pycode/batch.py


In [17]:
%%writefile docker-compose.yml

# Specifies the Docker Compose file version
version: '3.7'

# Defines the services that make up your app
services:
  # Defines a service name
  s3:
    # Specifies the Docker image to use for this service
    image: localstack/localstack
    # Maps ports between the host and the container
    ports:
      - "4566:4566"
    # Sets environment variables for the service
    environment:
      - SERVICES=s3

Writing docker-compose.yml


## Q5. Creating test data

Now let's create `integration_test.py`

We'll use the dataframe we created in Q3 (the dataframe for the unit test)
and save it to S3. You don't need to do anything else: just create a dataframe 
and save it.

We will pretend that this is data for January 2022.

Run the `integration_test.py` script. After that, use AWS CLI to verify that the 
file was created. 

Use this snipped for saving the file:

```python
df_input.to_parquet(
    input_file,
    engine='pyarrow',
    compression=None,
    index=False,
    storage_options=options
)
```

What's the size of the file?

- 3667
- 23667
- 43667
- 63667

Note: it's important to use the code from the snippet for saving
the file. Otherwise the size may be different depending on the OS,
engine and compression. Even if you use this exact snippet, the size
of your dataframe may still be a bit off. Just select the closest option.

In [18]:
%%writefile ./tests/integration_test.py

#!/usr/bin/env python
# coding: utf-8

import os
import sys

# Append the parent directory of the tests folder to the system path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from pycode import batch

import pandas as pd
from datetime import datetime


def dt(hour, minute, second=0):
    return datetime(2022, 1, 1, hour, minute, second)

 
def test_read_data() -> pd.DataFrame:
    """Read data into DataFrame"""
    data    = [
        (None, None, dt(1, 2), dt(1, 10)),
        (1, None, dt(1, 2), dt(1, 10)),
        (1, 2, dt(2, 2), dt(2, 3)),
        (None, 1, dt(1, 2, 0), dt(1, 2, 50)),
        (2, 3, dt(1, 2, 0), dt(1, 2, 59)),
        (3, 4, dt(1, 2, 0), dt(2, 2, 1)),     
    ]
    categorical = ['PULocationID', 'DOLocationID']   
    columns = ['PULocationID', 'DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime']

    df = pd.DataFrame(data, columns=columns)

    return df


S3_ENDPOINT_URL = os.getenv('S3_ENDPOINT_URL')
if S3_ENDPOINT_URL is not None:
    options = {
        'client_kwargs': {
            'endpoint_url': S3_ENDPOINT_URL
        }
    }
else:
    options = None
    

"""Read data into DataFrame"""
df_input = test_read_data()
input_file = batch.get_input_path(2022, 1)
output_file = batch.get_output_path(2022, 1)

print(df_input)

df_input.to_parquet(
    input_file,
    engine='pyarrow',
    compression=None,
    index=False,
    storage_options=options
)

# Actual Data
os.system('python pycode/batch.py 2022 1')
df_actual = pd.read_parquet(output_file, storage_options=options)

assert abs(df_actual['predicted_duration'].sum().round(2) - 31.51) < 0.1

Writing ./tests/integration_test.py


In [19]:
!python ./tests/integration_test.py

   PULocationID  DOLocationID tpep_pickup_datetime tpep_dropoff_datetime
0           NaN           NaN  2022-01-01 01:02:00   2022-01-01 01:10:00
1           1.0           NaN  2022-01-01 01:02:00   2022-01-01 01:10:00
2           1.0           2.0  2022-01-01 02:02:00   2022-01-01 02:03:00
3           NaN           1.0  2022-01-01 01:02:00   2022-01-01 01:02:50
4           2.0           3.0  2022-01-01 01:02:00   2022-01-01 01:02:59
5           3.0           4.0  2022-01-01 01:02:00   2022-01-01 02:02:01
Predict data: 100%|███████████████████████████████| 3/3 [00:00<00:00,  3.05it/s]
predicted mean duration: 10.5
predicted sum duration: 31.51


In [33]:
# So what's the size of file?
!stat ./output/yellow_tripdata_2022-01.parquet  | grep Size:       | awk '{print $2}'
!du -h ./output/yellow_tripdata_2022-01.parquet | awk '{print $1}'
!wc -c ./output/yellow_tripdata_2022-01.parquet

1995
4.0K
1995 ./output/yellow_tripdata_2022-01.parquet


In [34]:
%%writefile ./tests/integration_test.sh
#!/usr/bin/env bash

docker-compose up -d

sleep 5

export INPUT_FILE_PATTERN="s3://nyc-duration/in/{year:04d}-{month:02d}.parquet"
export OUTPUT_FILE_PATTERN="s3://nyc-duration/out/{year:04d}-{month:02d}.parquet"
export S3_ENDPOINT_URL="http://localhost:4566"


aws --endpoint-url="${S3_ENDPOINT_URL}" s3 mb s3://nyc-duration

pipenv run python integration_test.py

ERROR_CODE=$?

if [ ${ERROR_CODE} != 0 ]; then
    docker-compose logs
    docker-compose down
    exit ${ERROR_CODE}
fi

echo "yay tests work!"

docker-compose down

Overwriting ./tests/integration_test.sh


## Q6. Finish the integration test

We can read from our localstack s3, but we also need to write to it.

Create a function `save_data` which works similarly to `read_data`,
but we use it for saving a dataframe. 

Let's run the `batch.py` script for "January 2022" (the fake data
we created in Q5). 

We can do that from our integration test in Python: we can use
`os.system` for doing that (there are other options too). 

Now it saves the result to localstack.

The only thing we need to do now is to read this data and 
verify the result is correct. 

What's the sum of predicted durations for the test dataframe?

- 10.50
- 31.51
- 59.28
- 81.22

In [35]:
!python ./pycode/batch.py 2022 1

Predict data: 100%|███████████████████████████████| 3/3 [00:00<00:00,  3.33it/s]
predicted mean duration: 10.5
predicted sum duration: 31.51


## Running the test (ungraded)

The rest is ready, but we need to write a shell script for doing 
that. 

Let's do that!

## Submit the results

* Submit your results here: https://forms.gle/vi7k972SKLmpwohG8
* It's possible that your answers won't match exactly. If it's the case, select the closest one.
* You can submit your answers multiple times. In this case, the last submission will be used for scoring.

## Deadline

The deadline for submitting is 16 July (Sunday) 23:00 CEST. After that, the form will be closed.

# End of The Project